# Logistic Regression with pipeline

## Import packages

In [0]:
from databricks.automl_runtime.sklearn.column_selector import ColumnSelector
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from hyperopt import hp, tpe, fmin, STATUS_OK, Trials, SparkTrials
from optbinning import BinningProcess, OptimalBinning
from shap import KernelExplainer, summary_plot
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shap
import joblib

## User-defined functions
Functions include:
* `evaluateBinaryClassifier()`: returns KS score and decile of data.
* `model_diagnostics()`: returns the confusion matrix and model scores (accuracy, precision, recall, etc.).
* `psi_table()`: returns the population stability index of the in-time and OOT dataframes.
* `best_features()`: returns the `n` selected best features.
* `woe_map()`: returns the mapping of the binned variables.

In [0]:
def evaluateBinaryClassifier(predicted, actual, train_decile_reference, save=False, filename='validation.csv', setCutoff=False, new_data=False, probCutoff=0.5):
    tmp = pd.DataFrame({"probability" : predicted.values, "actual" : actual.values})

    if not new_data: tmp['bucket'] = pd.qcut(tmp['probability'], q=10, duplicates='drop')
    else:
        decile_bins, deciles = pd.qcut(train_decile_reference, 10, duplicates='drop', retbins=True)
        tmp['bucket'] = pd.cut(tmp['probability'], bins=deciles, labels=range(len(deciles)-1, 0, -1), right=False)
    
    tmp = tmp.groupby('bucket', as_index=True)

    tbl = pd.DataFrame()
    tbl['min_prob'] = tmp.min().probability.apply('{0:.7f}'.format)
    tbl['max_prob'] = tmp.max().probability.apply('{0:.7f}'.format)
    tbl['bad'] = tmp.sum().actual
    tbl['good'] = tmp.count().actual - tmp.sum().actual
    tbl['total'] = tmp.count().actual
    tbl = tbl.sort_values(by='min_prob', ascending=False).reset_index(drop=True)
    
    tbl['odds'] = (tbl['bad']/tbl['good']).apply('{0:.2f}'.format)
    tbl['bad_rate'] = (tbl['bad']/tbl['total']).apply('{0:.2f}'.format)
    tbl['ks'] = (np.round(((tbl['bad']/tbl['bad'].sum()).cumsum() - (tbl['good']/tbl['good'].sum()).cumsum()), 4)*100)
    
    tbl['max_ks'] = tbl.ks.apply(lambda x: "***" if x == tbl.ks.max() else "")
    maxdecile = tbl.reset_index().loc[tbl['ks'].idxmax()]

    if save: tbl.to_csv(filename)
    
    if not setCutoff:
        cutoff = float(tbl.loc[tbl.ks==tbl.ks.max()]['min_prob'].values[0])
        print("The recommended cutoff at decile", int(maxdecile['index'])+1, "is", cutoff, "with Kolmogorov-Smirnov Statistic equal to", maxdecile['ks'], "\n") 
        tbl.index +=1

        return cutoff, pd.DataFrame(tbl)
    
    else: 
        cutoff = probCutoff 
        print("Evaluating at defined cutoff equal to: ", cutoff) 
        tbl.index += 1   
        return pd.DataFrame(tbl)

def model_diagnostics(predicted, actual, cutoff):
    predicted = np.where(predicted >= cutoff, 1, 0)
    actual = actual.values
   
    cm = confusion_matrix(actual, predicted)
    tn, fp, fn, tp = cm.ravel()

    accuracy = round((tp+tn)/(tp+tn+fp+fn), 5)
    precision = round(tp/(tp+fp), 5)
    recall = round(tp/(tp+fn), 5)
    specificity = round(tn/(tn+fp), 5)
    f1 = round((2*(precision*recall))/(precision+recall), 2) 
    roc = round(roc_auc_score(actual, predicted), 5)
    gini = round(2*roc - 1, 5)
    
    metrics = {'accuracy' : accuracy, 'precision' : precision, 'recall' : recall, 
               'specificity' : specificity, 'f1' : f1, 'roc' : roc, 'gini' : gini}

    return cm, metrics

def psi_table(df1, df2):
    distribution_change = []
    psi_list = []

    for col in df1.columns.tolist():
        a = pd.DataFrame(df1[col].value_counts(normalize = True))
        b = pd.DataFrame(df2[col].value_counts(normalize = True))
        a.columns = ['a']
        b.columns = ['b']
        ab = a.join(b)
        ab['diff'] =  ab.a - ab.b
        ab['ln'] = np.log(ab.a/ab.b)
        ab['psi'] = ab['diff'] * ab['ln']
        psi = ab['psi'].sum()
        psi_list.append(round(psi,5))

        if psi < 0.1: distribution_change.append('No change')
        elif 0.1 <= psi <= 0.25: distribution_change.append('Slight change')
        else: distribution_change.append('Significant change')

    return pd.DataFrame({"columns" : supported_cols, "distribution change" : distribution_change, "psi" : psi_list})

def best_features(features, target, model, n_feats):
    importantFeats = RFE(model, n_features_to_select=n_feats)
    importantFeats = importantFeats.fit(features, target)

    return pd.DataFrame({"Feature" : features.columns.tolist(), "Important" : importantFeats.support_})

def woe_map(supported_cols, df, target_col):
  map = {}
  for variable in supported_cols:
    x = df[variable]
    y = df[target_col]

    if  x.values.dtype == object:
      data_type = 'categorical'
      bin_splits = x.unique().reshape(-1,1)

    else: 
      data_type = 'numerical'
      bin_splits = None

    optb = OptimalBinning(name = variable, dtype = data_type, monotonic_trend='descending', user_splits=bin_splits)
    optb.fit(x.values, y)
    binning_table = optb.binning_table
    df_binningtable = pd.DataFrame(binning_table.build())
    df_woe_table = df_binningtable[['Bin', 'WoE']].drop([len(df_binningtable['Bin'])-3,len(df_binningtable['Bin'])-2])

    map.update({variable : df_woe_table})

  return map

## Load data

In [0]:
df_loaded = spark.table('dsag_legacy_catalog.proj_pltodepo.aiml_fss_PL_final_02_modeling_base_imputed').toPandas()

df_loaded

## Select supported columns
Features are reordered based on data type for readability. Features are also selected based on IV, percentage of missing entries \
(\\(\approx\\)30%), number of unique entries (at least more than 1 for boolean or categorical data), and PSI.

**Caveats:**
* Columns `avg_ror_trans_l180d`, and `avg_ror_trans_l180d_bp` have ~ 36% missing entries but are imputed with 0.
* Feature `tenure` and `age` have a significant change in terms of PSI between In-time and OOT.

In [0]:
target_col = "dpd_31_ever"

bool_cols = ['with_depo_open', 'with_loan_ind',]
category_cols = ['num_children', 'num_yrs_in_company']
num_cols = ['balance_ytd', 'avg_ror_trans_l180d_bp', 'products_count', 'avg_ror_trans_l180d', 'avg_casa_adb_l180d', 'avg_casa_trans_l180d', ]

supported_cols = bool_cols + category_cols + num_cols
all_cols = [target_col] + supported_cols

#removed_cols
#bool_cols = ['with_loan_auto_ind', 'with_loan_home_ind', 'home_owned', ]
#category_cols = ['lifestage', 'num_yrs_in_present_address',]
#num_cols = ['age', 'tenure', ]

col_selector = ColumnSelector(supported_cols)

df_transformed = df_loaded[all_cols].copy()
df_transformed[bool_cols] = df_transformed[bool_cols].astype(object)

df_transformed

In [0]:
df_transformed.dtypes

## Train-test set split
Data is split into two parts:
* `X_train` - (60%) for training the model.
* `X_test` - (40%) for testing and prediction to compare with training set performance.

In [0]:
features = df_transformed[supported_cols]
target = df_transformed[target_col]

testing_size = 0.4
state = 0
#3 best results

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=testing_size, random_state=state, stratify=target)

## Preprocessing

Features are processed prior to modeling depending on data type.

### Boolean columns
Boolean features are binned and transformed in terms of WOE.

In [0]:
bool_params = {}
for col in bool_cols: bool_params.update({col : {"user_splits" : X_train[col].unique().reshape(-1,1)}})

bool_binner = BinningProcess(bool_cols, binning_fit_params=bool_params)

bool_pipeline = Pipeline([
    # ("cast_type", FunctionTransformer(lambda df: df.astype(object))),
    ("binner_bool", bool_binner),
])

bool_transformers = [("boolean", bool_pipeline, bool_cols)]

### Categorical columns
Categorical features are binned and transformed in terms of WOE.

In [0]:
category_params = {}
for col in category_cols: category_params.update({col : {"user_splits" : X_train[col].unique().reshape(-1,1)}})

category_binner = BinningProcess(category_cols, categorical_variables=category_cols, binning_fit_params=category_params)

category_pipeline = Pipeline([
    # ("cast_type", FunctionTransformer(lambda df: df.astype(object))),
    ("binner_cat", category_binner)
])

category_transformers = [("categorical", category_pipeline, category_cols)]

### Numerical columns
Numerical features are binned and transformed in terms of WOE with forced descending monotonic bins.

In [0]:
num_params = {}
for col in num_cols: num_params.update({col : {"monotonic_trend" : "descending"}})

num_binner = BinningProcess(num_cols, binning_fit_params=num_params)

numerical_pipeline = Pipeline([
    # ("converter", FunctionTransformer(lambda df: df.apply(pd.to_numeric, errors='coerce'))),
    ("binner_num", num_binner)
])

numerical_transformers = [("numerical", numerical_pipeline, num_cols)]

### Data transformation
Data is transformed prior to splitting and training.

In [0]:
transformers = bool_transformers + category_transformers + numerical_transformers

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

pipeline_val = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor)
    ])

pipeline_val.fit(X_train, y_train)

## Modeling

### Parameters

In [0]:
log_reg_params = {'random_state' : state,
                  'solver' : 'liblinear', # 'liblinear', 'lbfgs', 
                  'verbose' : 0,
                  "max_iter" : 10000,
                  }

### Trials

In [0]:
def log_reg(log_reg_params):
  log_model = LogisticRegression(**log_reg_params)

  model = Pipeline([
        ("column_selector", col_selector),
        ("preprocessor", preprocessor),
        ("classifier", log_model),
  ])

  model.fit(X_train, y_train)

  y_pred_test = model.predict(X_test)

  loss = recall_score(y_test, y_pred_test >= 0.3)

  return {
    'loss' : loss,
    'status' : STATUS_OK,
    'model_pipeline' : model,
  }

Use `SparkTrials()` for tuning along with the `hyperopt` module for fast trial iteration. Change back to `Trials()` after tuning.

Increase `max_evals` to at least 50 for many iterations to acquire best parameters. Change back to lower evaluations after tuning.

In [0]:
trials = SparkTrials()

best_hyperparams = fmin(fn = log_reg,
                        space = log_reg_params,
                        algo = tpe.suggest,
                        max_evals = 1,
                        show_progressbar = False,
                        trials = trials)

In [0]:
best_result = trials.best_trial["result"]
model_pipeline = best_result["model_pipeline"]
model = model_pipeline['classifier']
loss = best_result['loss']
print('Recall: ', round(loss,6))
model_pipeline

## Feature importance

Running `woe_map` to map the effects of the coefficients.

In [0]:
woe_map = woe_map(supported_cols, pd.concat([y_train, X_train], axis=1), target_col)

### Bar chart

In [0]:
feature_table = pd.DataFrame({"features":model_pipeline['preprocessor'].feature_names_in_,
                           "coefs":abs(model.coef_[0])}).sort_values("coefs")
coef_table = pd.DataFrame({"features":model_pipeline['preprocessor'].feature_names_in_,
                           "coefs":model.coef_[0]}).sort_values("coefs")

plt.figure(figsize=[8,12])
plt.subplot(2,1,1)
plt.title('Feature importance')
plt.barh(feature_table['features'], feature_table['coefs'])
plt.xlabel('Coefficient magnitude')
plt.subplot(2,1,2)
plt.title('Feature coefficients')
plt.barh(coef_table['features'], coef_table['coefs'])
plt.xlabel('Coefficient')
plt.show()

In [0]:
index = 0
print(supported_cols[index])
woe_map[supported_cols[index]]

In [0]:
coef_table.sort_index()

### Feature inference

**with_depo_open**
* 0 - not likely to default
* 1 - likely to default 

**with_loan_ind**
* 0 - not likely to default
* 1 - likely to default

**avg_casa_adb_l180d**
* `< 150,000` - not likely to default
* `> 150,000` - likely to default

**avg_casa_trans_l180d**
* `> 14,000` - not likely to default
* `< 14,000` - likely to default

**avg_ror_trans_l180d**
* `> 10,000` - not likely to default
* `< 10,000` - likely to default

**products_count**
* `> 2` - not likely to default
* `< 2` - likely to default

**num_yrs_in_company**
* `> 7` - not likely to default
* `< 7` - likely to default

**avg_ror_trans_l180d_bp**
* `> 7,000` - not likely to default
* `< 7,000` - likely to default

**num_children**
* 1+ - not likely to default
* 0 - likely to default

**balance_ytd**
* `> 47,000` - not likely to default
* `< 47,000` - likely to default

### Variance Inflation Factor

In [0]:
X_train_transformed = pd.DataFrame(model_pipeline['preprocessor'].transform(X_train))
X_test_transformed = pd.DataFrame(model_pipeline['preprocessor'].transform(X_test))
X_train_transformed.columns = supported_cols
X_test_transformed.columns = supported_cols

vif_data = pd.DataFrame() 
vif_data["feature"] = supported_cols 

vif_data["VIF"] = [vif(X_train_transformed.values, i) for i in range(len(supported_cols))] 
vif_data

### SHAP Values Plot

In [0]:
# X_train_sample = shap.sample(X_train, 200)
# X_test_sample = shap.sample(X_test, 200)
# predict = lambda x: model_pipeline.predict(pd.DataFrame(x, columns=X_train_sample.columns))
# explainer = shap.KernelExplainer(predict, X_train_sample, link='identity')
# shap_values = explainer.shap_values(X_test_sample, l1_reg=False)

linear_explainer = shap.LinearExplainer(model, X_train_transformed)
linear_shap_vals = linear_explainer(X_test_transformed)

In [0]:
shap.plots.beeswarm(linear_shap_vals, max_display=15)

In [0]:
# summary_plot(shap_values, X_test_sample, class_names=model.classes_, max_display=20, plot_type='bar')

In [0]:
# shap.plots.waterfall(linear_shap_vals[0])

### Recursive Feature Elimination

In [0]:
# best_features(X_train_transformed, y_train, model, 10)

## Inference

Evaluating the model using KS score and model diagnostics.

In [0]:
y_pred_train_prob_event = pd.DataFrame(model_pipeline.predict_proba(X_train)).loc[:,1]
y_pred_test_prob_event = pd.DataFrame(model_pipeline.predict_proba(X_test)).loc[:,1]

#### Training set

In [0]:
train_cutoff, train_evaluation = evaluateBinaryClassifier(y_pred_train_prob_event, y_train, y_pred_train_prob_event, save=False, filename='validation.csv', setCutoff=False, new_data=False, probCutoff=0.5)

train_evaluation

#### Test set evaluation

In [0]:
test_evaluation = evaluateBinaryClassifier(y_pred_test_prob_event, y_test, y_pred_train_prob_event, save=False, filename='validation.csv', setCutoff=True, new_data=True, probCutoff=train_cutoff)

test_evaluation

#### Metrics

In [0]:
cm_train, train_metrics = model_diagnostics(y_pred_train_prob_event, y_train, train_cutoff)
cm_test, test_metrics = model_diagnostics(y_pred_test_prob_event, y_test, train_cutoff)

model_metrics = pd.DataFrame({'train' : train_metrics, 'test' : test_metrics})
model_metrics

## OOT testing

In [0]:
df_loaded_oot = spark.table('dsag_legacy_catalog.proj_pltodepo.aiml_fss_pl_depo_cross_sell_02_modeling_base_oot').toPandas()
df_transformed_oot = df_loaded_oot[all_cols].copy()
df_transformed_oot

In [0]:
X_oot = df_transformed_oot.drop(target_col, axis=1)
y_oot = df_transformed_oot[target_col]
y_pred_oot_prob_event = pd.DataFrame(model_pipeline.predict_proba(X_oot)).loc[:,1]

In [0]:
oot_evaluation = evaluateBinaryClassifier(y_pred_oot_prob_event, y_oot, y_pred_train_prob_event, save=False, filename='validation.csv', setCutoff=True, new_data=True, probCutoff=train_cutoff)

oot_evaluation

### Final Metrics

In [0]:
cm_oot, oot_score = model_diagnostics(y_pred_oot_prob_event, y_oot, train_cutoff)
model_metrics['OOT'] = oot_score
model_metrics

## In-time vs OOT PSI comparison

In [0]:
psi_table(X_train, X_oot)

In [0]:
psi_table(features, X_oot)

In [0]:
psi_table(X_train, X_test)

## Exporting final model

In [0]:
joblib.dump(model_pipeline, "model_logreg_pipeline.pkl")

In [0]:
model_test = joblib.load("model_pipeline_test.pkl")

In [0]:
model_test.predict(df_transformed_oot)

In [0]:
model_test